In [24]:
import pandas as pd

In [25]:
parcel_type = pd.read_csv('RawData/Parcel_data.csv' , low_memory=False)

In [26]:
# subset the property tax data by multiresidential units (i.e R2 units) and for 2014
R2=(parcel_type['Property Class Code']=='A') | (parcel_type['Property Class Code']=='CO') \
| (parcel_type['Property Class Code']=='DA') | (parcel_type['Property Class Code']=='DD') \
| (parcel_type['Property Class Code']=='DF')| (parcel_type['Property Class Code']=='F') \
| (parcel_type['Property Class Code']=='FA')| (parcel_type['Property Class Code']=='F2') \
| (parcel_type['Property Class Code']=='OA') | (parcel_type['Property Class Code']=='TIA') \
| (parcel_type['Property Class Code']=='TIF') | (parcel_type['Property Class Code']=='XV')


parcel_type_r2 = parcel_type[R2]
parcel_type_r2_2014 = parcel_type_r2[parcel_type_r2['Closed Roll Fiscal Year'] ==2014 ]
#print(parcel_type.shape(), parcel_type_r2.shape())
parcel_type_r2_2014.reset_index(inplace=True)
#parcel_type_r2=parcel_type_r2.drop_duplicates(['Block and Lot Number']) 
parcel_type_r2_2014.shape

(25481, 43)

In [27]:
parcel_type_r2_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25481 entries, 0 to 25480
Data columns (total 43 columns):
index                                         25481 non-null int64
Closed Roll Fiscal Year                       25481 non-null float64
Property Location                             25481 non-null object
Neighborhood Code                             25481 non-null object
Neighborhood Code Definition                  25466 non-null object
Block and Lot Number                          25481 non-null object
Volume Number                                 25481 non-null int64
Property Class Code                           25481 non-null object
Property Class Code Definition                25481 non-null object
Year Property Built                           24732 non-null float64
Number of Bathrooms                           25481 non-null float64
Number of Bedrooms                            25481 non-null int64
Number of Rooms                               25481 non-null int64
Number o

In [4]:
# use this to clean other data frames
parcel_type_r2_2014[['Block and Lot Number','Property Location','Location']].to_csv('residential_units.csv')

In [28]:
parcel_type_r2_2014['neigh_code'] =parcel_type_r2_2014['Neighborhood Code'].str.extract('(\d+)', expand=False)
parcel_type_r2_2014['yr_built'] =parcel_type_r2_2014['Year Property Built'].replace(14,2014)
parcel_type_r2_2014['yr_built'].fillna(parcel_type_r2_2014['yr_built'].median(),inplace=True)
parcel_type_r2_2014['Construction Type'].fillna(parcel_type_r2_2014['Construction Type'].value_counts().idxmax(),inplace=True)
parcel_type_r2_2014['Lot Code'].fillna(parcel_type_r2_2014['Lot Code'].value_counts().idxmax(),inplace=True)
parcel_type_r2_2014['Tax Rate Area Code']=parcel_type_r2_2014['Tax Rate Area Code'].astype(str)
parcel_type_r2_2014['Tax Rate Area Code'].replace('nan',parcel_type_r2_2014['Tax Rate Area Code'].value_counts().idxmax(),inplace=True)
parcel_type_r2_2014['Closed Roll Assessed Fixtures Value'].unique() 

columns_to_use=['Block and Lot Number',
'Property Class Code'
,'yr_built'
,'neigh_code'
,'Number of Bedrooms'
,'Number of Bathrooms'
,'Number of Stories'
,'Number of Rooms'
,'Number of Units'
,'Construction Type'
,'Lot Depth'
,'Lot Frontage'
,'Basement Area'
,'Property Area in Square Feet'
,'Lot Area'
,'Lot Code'
,'Tax Rate Area Code' 
,'Percent of Ownership'
,'Closed Roll Assessed Fixtures Value'
,'Closed Roll Assessed Improvement Value'
,'Closed Roll Assessed Land Value'
,'Closed Roll Assessed Personal Prop Value'
,'Location']

parcel_main=parcel_type_r2_2014[columns_to_use]

/Users/taniajogesh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/taniajogesh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/taniajogesh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [29]:
parcel_main.shape

(25481, 23)

In [30]:
#checking that every row is a unique parcel number
print(parcel_type_r2_2014.drop_duplicates(['Block and Lot Number']).shape, parcel_type_r2_2014.shape)

((25481, 45), (25481, 45))


In [31]:
parcel_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25481 entries, 0 to 25480
Data columns (total 23 columns):
Block and Lot Number                        25481 non-null object
Property Class Code                         25481 non-null object
yr_built                                    25481 non-null float64
neigh_code                                  25481 non-null object
Number of Bedrooms                          25481 non-null int64
Number of Bathrooms                         25481 non-null float64
Number of Stories                           25481 non-null int64
Number of Rooms                             25481 non-null int64
Number of Units                             25481 non-null int64
Construction Type                           25481 non-null object
Lot Depth                                   25481 non-null float64
Lot Frontage                                25481 non-null float64
Basement Area                               25481 non-null float64
Property Area in Square Feet    

In [32]:
parcel_main_flat =pd.concat([parcel_main.drop('Property Class Code', axis=1), 
                             pd.get_dummies(parcel_main['Property Class Code'], prefix='prop_code')], axis=1)
parcel_main_flat =pd.concat([parcel_main_flat.drop('neigh_code', axis=1), 
                             pd.get_dummies(parcel_main['neigh_code'], prefix='neigh_code')], axis=1)
parcel_main_flat =pd.concat([parcel_main_flat.drop('Construction Type', axis=1), 
                             pd.get_dummies(parcel_main['Construction Type'], prefix='constr_code')], axis=1)
parcel_main_flat =pd.concat([parcel_main_flat.drop('Tax Rate Area Code', axis=1), 
                             pd.get_dummies(parcel_main['Tax Rate Area Code'], prefix='tax_code')], axis=1)
parcel_main_flat =pd.concat([parcel_main_flat.drop('Lot Code', axis=1), 
                             pd.get_dummies(parcel_main['Lot Code'], prefix='lot_code')], axis=1)

#pd.get_dummies(parcel_main['Property Class Code'])

In [33]:
parcel_main_flat.to_csv('parcel_data_clean.csv')

In [34]:
parcel_main_flat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25481 entries, 0 to 25480
Data columns (total 60 columns):
Block and Lot Number                        25481 non-null object
yr_built                                    25481 non-null float64
Number of Bedrooms                          25481 non-null int64
Number of Bathrooms                         25481 non-null float64
Number of Stories                           25481 non-null int64
Number of Rooms                             25481 non-null int64
Number of Units                             25481 non-null int64
Lot Depth                                   25481 non-null float64
Lot Frontage                                25481 non-null float64
Basement Area                               25481 non-null float64
Property Area in Square Feet                25481 non-null int64
Lot Area                                    25481 non-null float64
Percent of Ownership                        25481 non-null float64
Closed Roll Assessed Fixtures V